In [ ]:
import base64
import io
import asyncio
import logging

import pdfplumber
from dash import Dash, dcc, html, Input, Output, State, clientside_callback
import dash_bootstrap_components as dbc
from flask import Flask

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

# ---------------- HELPER: Extract PDF Text ----------------
def extract_pdf_text(pdf_data: bytes) -> str:
    """
    Extracts ALL text from a PDF using pdfplumber (no page limit),
    and returns the combined string. If pdfplumber fails, returns
    an error message.
    """
    try:
        with pdfplumber.open(io.BytesIO(pdf_data)) as pdfplumb:
            pages_text = []
            for page in pdfplumb.pages:
                page_text = page.extract_text() or ""
                pages_text.append(page_text)
            return "\n".join(pages_text)
    except Exception as e:
        logger.error(f"Error reading PDF with pdfplumber: {e}")
        return "[Error reading PDF text]"


# -------------- FILE UPLOAD CALLBACK ----------------
def register_file_upload_callback(app, name):
    @app.callback(
        Output('uploaded-file' + name, component_property='data'),
        Input('upload-data' + name, component_property='contents'),
        State('upload-data' + name, component_property='filename'),
        State('uploaded-file' + name, component_property='data')
    )
    def _upload_files(contents, filenames, current_data):
        """
        - contents: list of base64 data URIs from dcc.Upload
        - filenames: list of str
        - current_data: (encoded_files, file_texts, file_names) already stored
          from previous uploads

        We store:
        (a) encoded_files -> just the *original* data URI from Dash for display
        (b) file_texts -> the extracted text (PDF or plain) for doc QA
        (c) file_names -> the original filenames
        """
        encoded_files = current_data[0] if current_data else []
        file_texts    = current_data[1] if current_data else []
        file_names    = current_data[2] if current_data else []

        if not contents:
            return encoded_files, file_texts, file_names

        for content, filename in zip(contents, filenames):
            # content looks like "data:application/pdf;base64,<BASE64_STR>"
            if not filename:
                continue

            # For simple extension check:
            ext = filename.lower().split('.')[-1]

            if ext == 'pdf':
                logger.info(f"Processing PDF: {filename}")
                # 1) Decode just for text extraction
                try:
                    # Split out "data:application/pdf;base64," prefix
                    _, b64_str = content.split(",", 1)
                    pdf_data = base64.b64decode(b64_str)
                    pdf_text = extract_pdf_text(pdf_data)
                except Exception as e:
                    logger.error(f"Failed to decode PDF: {e}")
                    pdf_text = "[Error decoding PDF]"

                file_texts.append(pdf_text)
                # 2) For display, store the *original* content from dash
                encoded_files.append(content)
                file_names.append(filename)

            elif ext == 'txt':
                logger.info(f"Processing TXT: {filename}")
                # Split out "data:text/plain;base64," prefix
                try:
                    _, b64_str = content.split(",", 1)
                    txt_data = base64.b64decode(b64_str).decode(errors='replace')
                except Exception as e:
                    logger.error(f"Failed to decode TXT: {e}")
                    txt_data = "[Error decoding TXT]"
                file_texts.append(txt_data)
                encoded_files.append(content)  # data:text/plain;base64,...
                file_names.append(filename)

            else:
                logger.warning(f"Skipping unsupported file type: {filename}")

        return encoded_files, file_texts, file_names


# ---------------- TABS / IFRAME CALLBACK ----------------
def register_update_tabs_callback(app, name):
    @app.callback(
        Output('file_tabs' + name, 'tabs'),
        Output('file_tabs' + name, 'children'),
        Input('uploaded-file' + name, 'data'),
        Input('file_tabs' + name, 'active_tab'),
    )
    def update_tabs(data, active_tab):
        """
        We build dynamic tabs based on the files. For each tab, we show
        an <iframe> (or <object>) to display the PDF or text.
        """
        if not data or not data[0]:
            return [], []

        encoded_files, file_texts, file_names = data
        num_files = len(encoded_files)

        # Build a list of tab definitions
        tabs = [
            {'id': f'tab{i + 1}', 'title': file_names[i]}
            for i in range(num_files)
        ]

        # If no active tab, default to tab1
        if not active_tab:
            active_tab = "tab1" + name

        # The active tab ID will look like "tab1AWM" => remove "tab" + name
        tab_index_str = active_tab.replace("tab", "").replace(name, "")
        try:
            tab_id = int(tab_index_str) - 1
        except ValueError:
            tab_id = 0

        # Ensure we don't go out of range
        if tab_id < 0 or tab_id >= num_files:
            return tabs, []

        # We have the selected file data
        file_uri = encoded_files[tab_id]  # "data:application/pdf;base64,..." or text
        # We'll embed in an Iframe:
        children = [
            html.Iframe(
                id=f'output-data-upload-{tab_id}' + name,
                src=file_uri,
                height='1000px',
                width='100%',
            )
        ]

        return tabs, children


# ---------------- CONVERSATION UI ----------------
def textbox(text, box="AI", name="Philippe"):
    style = {
        "maxWidth": "60%",
        "width": "max-content",
        "padding": "5px 10px",
        "borderRadius": 25,
        "marginBottom": 20,
    }

    if box == "user":
        style["marginLeft"] = "auto"
        style["marginRight"] = "10px"
        style["marginTop"] = "10px"
        return dbc.Card(text, style=style, body=True, color="primary", inverse=True)

    elif box == "AI":
        style["marginLeft"] = "10px"
        style["marginRight"] = "auto"
        style["marginTop"] = "10px"
        from dash import dcc, html
        markdown = dcc.Markdown(text)
        return html.Div([
            dbc.Card(
                children=[markdown],
                style=style,
                body=True,
                color="light",
                inverse=False
            )
        ])

    raise ValueError("Incorrect 'box' argument.")


def register_update_display_callback(app, name):
    from dash import html

    @app.callback(
        Output("display-conversation" + name, "children"),
        Input("store-conversation" + name, "data"),
    )
    def update_display(chat_history):
        if not chat_history:
            return html.Div(
                html.Img(
                    src=app.get_asset_url('Logo.png'),
                    style={
                        'display': 'flex',
                        'justifyContent': 'center',
                        'alignItems': 'center',
                        'height': '200px',
                        'width': '200px'
                    }
                ),
                style={
                    'display': 'flex',
                    'justifyContent': 'center',
                    'alignItems': 'center',
                    'height': '80vh'
                }
            )

        # Each message is separated by <split>
        messages = [m for m in chat_history.split("<split>") if m.strip()]

        # Alternate user/AI
        chat_bubbles = []
        for i, msg in enumerate(messages):
            if i % 2 == 0:
                # user bubble
                chat_bubbles.append(textbox(msg, box="user"))
            else:
                # AI bubble
                chat_bubbles.append(textbox(msg, box="AI"))

        return chat_bubbles


def register_clean_input_callback(app, name):
    @app.callback(
        Output("chat-input" + name, "value"),
        Input("submit" + name, "n_clicks"),
        Input("chat-input" + name, "n_submit")
    )
    def clear_input(n_clicks, n_submit):
        return ""


# --------------- LLM LOGIC EXAMPLE ---------------
async def get_bot_real_answer(input_value, file_content, llm_type):
    """
    Dummy async function that would call your LLM. 
    - 'file_content' is a list of extracted text from the PDF(s) or .txt(s).
    """
    # Build a prompt or do your doc QA. We'll just fake it:
    return f"(AI answer to: {input_value})"


def register_run_chatbot_callback(app, name):
    @app.callback(
        [Output("store-conversation" + name, "data"),
         Output("loading-component" + name, "children"),
         Output("llm-type" + name, "disabled")],
        [Input("submit" + name, "n_clicks"),
         Input("chat-input" + name, "n_submit")],
        [State("chat-input" + name, "value"),
         State("store-conversation" + name, "data"),
         State("uploaded-file" + name, "data"),
         State("llm-type" + name, "value")],
    )
    def run_chatbot(n_clicks, n_submit, user_input, chat_history, file_content, llm_type):
        if (n_clicks is None or n_clicks == 0) and (n_submit is None):
            return "", None, False

        if not user_input:
            return chat_history, None, False

        if not chat_history:
            chat_history = ""

        # file_content => [encoded_files, file_texts, file_names]
        # The text from PDFs/TXTs is in file_content[1]
        texts = file_content[1] if file_content else []

        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
        model_output = loop.run_until_complete(get_bot_real_answer(user_input, texts, llm_type))

        # Append conversation
        chat_history += f"You: {user_input}<split>Philippe: {model_output}<split>"
        return chat_history, None, True


# ---------------- LAYOUT & APP SETUP ----------------
def get_conversations_html(name):
    return html.Div(
        children=[
            dbc.Select(
                id="llm-type" + name,
                value='gemini-1.5-pro-text',
                options=[
                    {'label': 'Gemini Flash', 'value': 'gemini-1.5-flash-text'},
                    {'label': 'Gemini Pro', 'value': 'gemini-1.5-pro-text'}
                ],
            ),
            html.Div(id="display-conversation" + name),
        ],
        style={
            "overflow": "auto",
            "display": "flex",
            "height": "85vh",
            "flexDirection": "column",
            "backgroundColor": "#eeeeee"
        }
    )

def get_controls_html(name):
    return dbc.InputGroup(
        children=[
            dbc.Input(id="chat-input" + name, placeholder="Message", type="text"),
            dbc.Button("Submit", id="submit" + name),
        ]
    )

def setup_layout(name, ENV="DEV"):
    """
    Example layout using your existing pattern with
    a file upload area, dynamic tabs to display the PDF,
    and conversation UI on the side.
    """
    import uitoolkit_plotly_dash as uitk  # If you're using your custom module
    return html.Div(style={'width': '100%', 'height': '100%', 'margin': 0, 'padding': 0, 'overflow': 'hidden'}, children=[
        uitk.HeaderAllIn(
            menu_items=[{'type': 'outlined', 'id': "unique_id" + name}],
            brand={'appName': f'{name} BYOD', 'envBadge': {'name': f'{ENV}'}},
            search_expand_mode='none'
        ),
        html.Div(id='page-load-div' + name, style={'display': 'none'}),

        html.Div(children=[
            uitk.Row(children=[
                uitk.Col(children=[
                    html.Div(children=[
                        dcc.Upload(
                            id='upload-data' + name,
                            children=html.Div(['Drag and Drop or ', html.A('Select Files (.txt, .pdf)')]),
                            style={
                                'width': '100%',
                                'height': '60px',
                                'lineHeight': '60px',
                                'borderWidth': '1px',
                                'borderStyle': 'dashed',
                                'borderRadius': '5px',
                                'textAlign': 'center',
                                'margin': '10px'
                            },
                            multiple=True
                        ),
                        uitk.DynamicTabs(id="file_tabs" + name, active_tab="tab1" + name, tabs=[{}]),
                    ]),
                ]),
                uitk.Col(children=[
                    dbc.Container(
                        children=[
                            dcc.Store(id="store-conversation" + name, data=""),
                            get_conversations_html(name),
                            get_controls_html(name),
                            dbc.Spinner(html.Div(id="loading-component" + name))
                        ]
                    )
                ], style={'width': 'auto'}),
            ], style={'padding': '10px'}),
            html.Div(id='hidden-div' + name, style={'display': 'none'}),
            uitk.Spinner(html.Div(
                id='storing-file' + name,
                children=[dcc.Store(id='uploaded-file' + name)],
                style={'minHeight': '50px'}
            ))
        ])
    ])


# ---------- FLASK & DASH SERVER SETUP -----------
server = Flask(__name__)

# Two separate apps for demonstration
app_awm = Dash(
    __name__, 
    update_title="AWM AI Strats", 
    url_base_pathname='/awm-byod/', 
    server=server
)

app_cfo = Dash(
    __name__, 
    update_title="CFO AI Strats", 
    url_base_pathname='/cfo-byod/', 
    server=server
)

APP_AWM_NAME = 'AWM'
APP_CFO_NAME = 'CFO'

# Layouts
app_awm.layout = setup_layout(APP_AWM_NAME)
app_cfo.layout = setup_layout(APP_CFO_NAME)

# Optional: clientside callbacks for page loading
app_awm.clientside_callback(
    clientside_callback.ClientsideFunction(
        namespace='clientside', 
        function_name='fetch_authn'
    ),
    Output('page-load-div' + APP_AWM_NAME, 'children'),
    Input('page-load-div' + APP_AWM_NAME, 'id')
)

app_cfo.clientside_callback(
    clientside_callback.ClientsideFunction(
        namespace='clientside', 
        function_name='fetch_authn'
    ),
    Output('page-load-div' + APP_CFO_NAME, 'children'),
    Input('page-load-div' + APP_CFO_NAME, 'id')
)

# Register all your custom Python callbacks
register_file_upload_callback(app_awm, APP_AWM_NAME)
register_update_tabs_callback(app_awm, APP_AWM_NAME)
register_update_display_callback(app_awm, APP_AWM_NAME)
register_clean_input_callback(app_awm, APP_AWM_NAME)
register_run_chatbot_callback(app_awm, APP_AWM_NAME)

register_file_upload_callback(app_cfo, APP_CFO_NAME)
register_update_tabs_callback(app_cfo, APP_CFO_NAME)
register_update_display_callback(app_cfo, APP_CFO_NAME)
register_clean_input_callback(app_cfo, APP_CFO_NAME)
register_run_chatbot_callback(app_cfo, APP_CFO_NAME)


if __name__ == '__main__':
    server.run(debug=True, host='0.0.0.0', port=8050)


In [ ]:
import base64
import io
import asyncio
import logging

import pdfplumber
from PyPDF2 import PdfReader  # optional if you need further PyPDF2 usage
from dash.dependencies import Input, Output, State
from dash import Dash, dcc, html
import dash_bootstrap_components as dbc
from flask import Flask

# -------------- LOGGER SETUP -------------
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

# ---------------- HELPERS ----------------
def process_pdf(decoded_pdf: bytes) -> tuple[str, str]:
    """
    Extracts ALL text from a PDF using pdfplumber (no page limit),
    and returns (pdf_text, pdf_base64), where pdf_base64 is the
    entire original PDF base64-encoded for display in an iframe.
    """
    extracted_text = ""
    try:
        with pdfplumber.open(io.BytesIO(decoded_pdf)) as pdfplumb:
            all_text = []
            for page in pdfplumb.pages:
                # Extract page text (None => "")
                page_text = page.extract_text() or ""
                all_text.append(page_text)
            extracted_text = "\n".join(all_text)
    except Exception as e:
        logger.error(f"Error reading PDF: {e}")
        extracted_text = "[Error reading PDF text]"

    # For display in an iframe, we just encode the entire original PDF
    encoded_pdf = base64.b64encode(decoded_pdf).decode()
    return extracted_text, encoded_pdf

# ------------- YOUR CALLBACKS -------------
def register_file_upload_callback(app, name):
    @app.callback(
        Output("uploaded-file" + name, component_property="data"),
        Input("upload-data" + name, component_property="contents"),
        State("upload-data" + name, component_property="filename"),
        State("uploaded-file" + name, component_property="data"),
    )
    def _upload_files(contents, filenames, current_data):
        """
        Process user-uploaded files (PDF or TXT) and store:
         - encoded_files: base64 for iFrame display
         - file_texts: extracted text for QA
         - file_names: the original filenames
        """
        encoded_files = current_data[0] if current_data else []
        file_texts    = current_data[1] if current_data else []
        file_names    = current_data[2] if current_data else []

        if not contents:
            return encoded_files, file_texts, file_names

        for content, filename in zip(contents, filenames):
            content_type, content_string = content.split(",")

            try:
                decoded = base64.b64decode(content_string)
            except Exception as e:
                logger.error(f"Error decoding base64 for {filename}: {e}")
                continue

            # --- PDF Files ---
            if filename.lower().endswith(".pdf"):
                logger.info(f"Processing PDF: {filename}")
                pdf_text, pdf_b64 = process_pdf(decoded)
                file_texts.append(pdf_text)
                # For iFrame usage:
                encoded_files.append("data:application/pdf;base64," + pdf_b64)
                file_names.append(filename)

            # --- TXT Files ---
            elif filename.lower().endswith(".txt"):
                logger.info(f"Processing text file: {filename}")
                text_str = decoded.decode(errors="replace")
                file_texts.append(text_str)
                # For iFrame usage, base64-encode or store as data URI
                encoded_text = base64.b64encode(decoded).decode()
                encoded_files.append("data:text/plain;base64," + encoded_text)
                file_names.append(filename)

            else:
                logger.warning(f"Skipping unsupported file type: {filename}")
                continue

        return encoded_files, file_texts, file_names


def register_update_tabs_callback(app, name):
    @app.callback(
        Output("file_tabs" + name, component_property="tabs"),
        Output("file_tabs" + name, component_property="children"),
        Input("uploaded-file" + name, component_property="data"),
        Input("file_tabs" + name, component_property="active_tab"),
    )
    def update_tabs(data, active_tab):
        """
        Build dynamic tabs for uploaded files, each containing an iFrame 
        that displays the file content (base64-encoded PDF/TXT).
        """
        if not data or not data[0]:
            return [], []

        encoded_files, file_contents, file_names = data
        # Build the tab definitions
        tabs = [
            {"id": f"tab{i+1}", "title": file_names[i]}
            for i in range(len(encoded_files))
        ]

        if active_tab is None:
            # No active tab yet, default to tab1 if available
            active_tab = "tab1" + name

        # Extract the integer index from active tab
        # "tab2AWM" => we remove 'tab' and 'AWM', leaving just '2'
        tab_index_str = active_tab.replace("tab", "").replace(name, "")
        if not tab_index_str.isdigit():
            # fallback to 1
            tab_id = 0
        else:
            tab_id = int(tab_index_str) - 1

        # Construct an Iframe for the active tab
        if 0 <= tab_id < len(encoded_files):
            children = [
                html.Iframe(
                    id=f"output-data-upload-{tab_id}" + name,
                    src=encoded_files[tab_id],
                    height="1000px",
                    width="100%",
                )
            ]
        else:
            children = []

        return tabs, children


def textbox(text, box="AI", name="Philippe"):
    """
    Utility for rendering conversation messages with styling.
    """
    import dash_bootstrap_components as dbc
    from dash import dcc, html

    text = text.replace(f"{name}:", "").replace("You:", "")
    style = {
        "maxWidth": "60%",
        "width": "max-content",
        "padding": "5px 10px",
        "borderRadius": 25,
        "marginBottom": 20,
    }

    if box == "user":
        style["marginLeft"] = "auto"
        style["marginRight"] = "10px"
        style["marginTop"] = "10px"
        return dbc.Card(text, style=style, body=True, color="primary", inverse=True)

    elif box == "AI":
        style["marginLeft"] = "10px"
        style["marginRight"] = "auto"
        style["marginTop"] = "10px"
        textbox_markdown = dcc.Markdown(text)
        return html.Div([
            dbc.Card(children=[textbox_markdown], style=style, body=True, color="light", inverse=False)
        ])
    else:
        raise ValueError("Incorrect option for 'box'.")


def register_update_display_callback(app, name):
    from dash import html

    @app.callback(
        Output("display-conversation" + name, component_property="children"),
        [Input("store-conversation" + name, component_property="data")]
    )
    def update_display(chat_history):
        """
        Displays the conversation messages from our store.
        """
        if not chat_history:
            return html.Div(
                html.Img(src=app.get_asset_url('Logo.png'), 
                         style={'display': 'flex',
                                'justifyContent': 'center',
                                'alignItems': 'center',
                                'height': '200px',
                                'width': '200px'}),
                style={'display': 'flex', 'justifyContent': 'center', 'alignItems': 'center', 'height': '80vh'}
            )

        # Split at <split>
        messages = chat_history.split("<split>")
        # Last element might be empty
        messages = [msg for msg in messages if msg.strip()]

        # Alternate user / AI
        result = []
        for i, msg in enumerate(messages):
            if i % 2 == 0:
                # user
                result.append(textbox(msg, box="user"))
            else:
                # AI
                result.append(textbox(msg, box="AI"))
        return result


def register_clean_input_callback(app, name):
    @app.callback(
        Output("chat-input" + name, component_property="value"),
        [Input("submit" + name, component_property="n_clicks"),
         Input("chat-input" + name, component_property="n_submit")]
    )
    def clear_input(n_clicks, n_submit):
        """
        Clear the text input after user hits "Submit" or presses Enter.
        """
        return ""


# Example doc-qa logic: async fetching from an LLM
async def get_bot_real_answer(input_value, file_content, llm_type):
    """
    Example asynchronous function to get the bot's answer
    based on the user input and the combined text from uploaded files.
    """
    prompt = ""
    for k, doc in enumerate(file_content):
        if doc and len(prompt) < 500000 and k >= 0:
            prompt += f"Document number {k + 1}: {doc}\n"

    # This is just a placeholder. Replace with your actual model logic
    # e.g.: user_model = get_user_model(llm_type)
    # bot_answer = await user_model.agenerate(...)

    # For demonstration, we just echo back the user query
    bot_answer = f"(Pretend AI response to '{input_value}')"
    return bot_answer


def register_run_chatbot_callback(app, name):
    @app.callback(
        [Output("store-conversation" + name, component_property="data"),
         Output("loading-component" + name, component_property="children"),
         Output("llm-type" + name, component_property="disabled")],
        [Input("submit" + name, component_property="n_clicks"),
         Input("chat-input" + name, component_property="n_submit")],
        [State("chat-input" + name, component_property="value"),
         State("store-conversation" + name, component_property="data"),
         State("uploaded-file" + name, component_property="data"),
         State("llm-type" + name, component_property="value")]
    )
    def run_chatbot(n_clicks, n_submit, user_input, chat_history, file_content, llm_type):
        """
        Chatbot logic: appends user input + AI response to the conversation store.
        """
        # no user action yet
        if (n_clicks == 0 or n_clicks is None) and (n_submit is None):
            return "", None, False

        # empty user message => do nothing
        if not user_input:
            return chat_history, None, False

        # Ensure file_content is not None
        file_content = file_content if file_content else [[], [], []]
        # file_content[1] is the actual text from the PDF(s)/TXT(s).
        uploaded_texts = file_content[1] if len(file_content) > 1 else []

        # Run your model asynchronously
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
        model_output = loop.run_until_complete(
            get_bot_real_answer(user_input, uploaded_texts, llm_type))

        if not chat_history:
            chat_history = ""

        # Append
        chat_history += f"You: {user_input}<split>Philippe: {model_output}<split>"
        return chat_history, None, True


# ------------- LAYOUT & APP INIT -------------
def get_conversations_html(name):
    return html.Div(
        children=[
            dbc.Select(
                id="llm-type" + name,
                value='gemini-1.5-pro-text',
                options=[
                    {'label': 'Gemini Flash', 'value': 'gemini-1.5-flash-text'},
                    {'label': 'Gemini Pro', 'value': 'gemini-1.5-pro-text'}
                ],
            ),
            html.Div(id="display-conversation" + name),
        ],
        style={
            "overflow": "auto",
            "display": "flex",
            "height": "85vh",
            "flexDirection": "column",
            "backgroundColor": "#eeeeee",  # grey background
        },
    )

def get_controls_html(name):
    return dbc.InputGroup(
        children=[
            dbc.Input(id="chat-input" + name, placeholder="Message", type="text"),
            dbc.Button(children="Submit", id="submit" + name),
        ]
    )

def setup_layout(name, ENV="DEV"):
    import uitoolkit_plotly_dash as uitk  # assuming you have this module
    return html.Div(
        style={'width': '100%', 'height': '100%', 'margin': 0, 'padding': 0, 'overflow': 'hidden'},
        children=[
            uitk.HeaderAllIn(
                menu_items=[{'type': 'outlined', 'id': "unique_id" + name}],
                brand={'appName': f'{name} BYOD', 'envBadge': {'name': f'{ENV}'}},
                search_expand_mode='none'
            ),
            html.Div(id='page-load-div' + name, style={'display': 'none'}),
            html.Div(
                children=[
                    uitk.Row(
                        children=[
                            uitk.Col(
                                children=[
                                    html.Div(
                                        children=[
                                            dcc.Upload(
                                                id='upload-data' + name,
                                                children=html.Div([
                                                    'Drag and Drop or ',
                                                    html.A('Select Files (.txt, .pdf)')
                                                ]),
                                                style={
                                                    'width': '100%',
                                                    'height': '60px',
                                                    'lineHeight': '60px',
                                                    'borderWidth': '1px',
                                                    'borderStyle': 'dashed',
                                                    'borderRadius': '5px',
                                                    'textAlign': 'center',
                                                    'margin': '10px'
                                                },
                                                multiple=True
                                            ),
                                            uitk.DynamicTabs(id="file_tabs" + name, active_tab="tab1" + name, tabs=[{}]),
                                        ]
                                    )
                                ]
                            ),
                            uitk.Col(
                                children=[
                                    dbc.Container(
                                        children=[
                                            dcc.Store(id="store-conversation" + name, data=""),
                                            get_conversations_html(name),
                                            get_controls_html(name),
                                            dbc.Spinner(html.Div(id="loading-component" + name)),
                                        ]
                                    )
                                ],
                                style={'width': 'auto'},
                            ),
                        ],
                        style={'padding': '10px'},
                    ),
                    html.Div(id='hidden-div' + name, style={'display': 'none'}),
                    uitk.Spinner(
                        html.Div(
                            id='storing-file' + name,
                            children=[dcc.Store(id='uploaded-file' + name)],
                            style={'minHeight': '50px'}
                        )
                    ),
                ]
            ),
        ]
    )

# ---------------- MAIN INIT -------------
server = Flask(__name__)
app_awm = Dash(
    __name__, 
    update_title="AWM AI Strats",
    url_base_pathname='/awm-byod/', 
    server=server
)
app_cfo = Dash(
    __name__, 
    update_title="CFO AI Strats",
    url_base_pathname='/cfo-byod/', 
    server=server
)

APP_AWM_NAME = 'AWM'
APP_CFO_NAME = 'CFO'

# Layout
app_awm.layout = setup_layout(APP_AWM_NAME, ENV="DEV")
app_cfo.layout = setup_layout(APP_CFO_NAME, ENV="DEV")

# Register clientside callbacks for page loading, etc.
app_awm.clientside_callback(
    dash.clientside_callback.ClientsideFunction(namespace='clientside', function_name='fetch_authn'),
    Output('page-load-div' + APP_AWM_NAME, component_property='children'),
    Input('page-load-div' + APP_AWM_NAME, component_property='id')
)

app_cfo.clientside_callback(
    dash.clientside_callback.ClientsideFunction(namespace='clientside', function_name='fetch_authn'),
    Output('page-load-div' + APP_CFO_NAME, component_property='children'),
    Input('page-load-div' + APP_CFO_NAME, component_property='id')
)

# Register our Python callbacks
register_file_upload_callback(app_awm, APP_AWM_NAME)
register_update_tabs_callback(app_awm, APP_AWM_NAME)
register_update_display_callback(app_awm, APP_AWM_NAME)
register_clean_input_callback(app_awm, APP_AWM_NAME)
register_run_chatbot_callback(app_awm, APP_AWM_NAME)

register_file_upload_callback(app_cfo, APP_CFO_NAME)
register_update_tabs_callback(app_cfo, APP_CFO_NAME)
register_update_display_callback(app_cfo, APP_CFO_NAME)
register_clean_input_callback(app_cfo, APP_CFO_NAME)
register_run_chatbot_callback(app_cfo, APP_CFO_NAME)

if __name__ == '__main__':
    # Run your server
    server.run(debug=True, host='0.0.0.0', port=8050)
